# Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import math
from tqdm import tqdm
from backtesting import Strategy, Backtest
from backtesting.lib import resample_apply
from backtesting.lib import plot_heatmaps
from blist import sortedlist
from collections import deque

/home/ehsan/.local/lib/python3.8/site-packages/backtesting/_plotting.py:47: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

# Configs

In [2]:
DATA_PATH = './data/2019-2021_1m_BTC_ohlcv.csv'

class GeneralConfig:
    def __init__(self):
        self.close = 'close'
        self.open = 'open'
        self.high = 'high'
        self.low = 'low'
        self.volume = 'volume'
        self.time_stamp = 'open_time'
        self.time_stamp_unit = 'ms'
        self.buy = 'buy'
        self.stop_buy = 'stop_buy'
        self.stop_sell = 'stop_sell'
        self.sell = 'sell'
        self.short = 'short'
        self.long = 'long'
        self.not_sure = 'not_sure'


gen = GeneralConfig()


# Utils

In [9]:
def add_alpha(df, window_size, use_high=False):
    feature_name = 'alpha' + str(window_size)
    if not use_high:
        price_list = list(df[gen.close])
    else:
        price_list = list(df[gen.high])
    
    alpha_list = list()
    window = deque()
    sorted_window = sortedlist()  # keeps sorted prices in binary search tree for speed up

    for price in tqdm(price_list):
        window.append(price)
        sorted_window.add(price)
        if len(window) <= window_size:
            continue
        old_price = window.popleft()
        sorted_window.remove(old_price)

        alpha = (sorted_window.index(price) + sorted_window.count(price) - 1) / len(window)
        alpha_list.append(alpha)

    df[feature_name] = [-1] * window_size + alpha_list
    return df

# Data preparation

In [10]:
df = pd.read_csv(DATA_PATH)

In [11]:
df = add_alpha(df, 7000, use_high=True)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1051000/1051000 [00:25<00:00, 40657.69it/s]


In [14]:
df.head()

,exchange,symbol,open_time,open,high,low,close,volume,alpha7000,Close,Open,High,Low
0,Binance,BTC/USDT,1546300800000,3701.23,3703.72,3701.09,3702.46,17.100110,-1.0,3702.46,3701.23,3703.72,3701.09
1,Binance,BTC/USDT,1546300860000,3702.44,3702.63,3695.66,3697.04,23.700604,-1.0,3697.04,3702.44,3702.63,3695.66
2,Binance,BTC/USDT,1546300920000,3699.42,3702.04,3696.08,3698.14,14.488615,-1.0,3698.14,3699.42,3702.04,3696.08
3,Binance,BTC/USDT,1546300980000,3697.49,3698.19,3695.97,3696.51,8.499966,-1.0,3696.51,3697.49,3698.19,3695.97
4,Binance,BTC/USDT,1546301040000,3697.20,3697.62,3695.00,3696.32,21.782886,-1.0,3696.32,3697.20,3697.62,3695.00


In [ ]:




def set_df_schema(df):
    df['Close'] = df['close']
    df['Open'] = df['open']
    df['High'] = df['high']
    df['Low'] = df['low']
    df = df.ffill()
    return df[['Open', 'Close', 'High', 'Low', 'alpha7000']]


def SMA(array, n):
    """Simple moving average"""
    return pd.Series(array).rolling(n).mean()


def RSI(array, n):
    """Relative strength index"""
    # Approximate; good enough
    gain = pd.Series(array).diff()
    loss = gain.copy()
    gain[gain < 0] = 0
    loss[loss > 0] = 0
    rs = gain.ewm(n).mean() / loss.abs().ewm(n).mean()
    return 100 - 100 / (1 + rs)


class System(Strategy):
    alpha_long_multi = 0.75  # Daily RSI lookback periods
    alpha_short_multi = 1  # Weekly
    level_sell = 0.6
    level_buy = 1.1
    lock = False

    def init(self):
        self.alpha = self.data.alpha7000

    def next(self):
        price = self.data.Close[-1]

        # If we don't already have a position, and
        # if all conditions are satisfied, enter long.
        if (not self.position) and self.data.alpha7000[-1] > 0.98 and not self.lock:

            # Buy at market price on next open, but do
            # set 8% fixed stop loss.
            # if self.orders.__len__() > 0:
            #     self.orders[0].cancel()
            self.buy(sl=price*0.92)

        # If the price closes 2% or more below 10-day MA
        # close the position, if any.
        elif self.data.alpha7000[-1] < 0.02:
            self.position.close()
            self.lock = False



backtest = Backtest(set_df_schema(df), System, commission=.001, cash=100000)
backtest.run()


# stats, heatmap = backtest.optimize(
#     alpha_long_multi=[x/20 for x in range(10, 35, 5)],
#     alpha_short_multi=[x/20 for x in range(10, 35, 5)],
#     level_buy=[x/100 for x in range(10, 200, 50)],
#     level_sell=[x / 100 for x in range(10, 200, 50)],
#     constraint=lambda p: p.level_buy > p.level_sell,
#     max_tries=200,
#     random_state=0,
#     return_heatmap=True
# )

backtest.plot()
# plot_heatmaps(heatmap, agg='max')
# print(heatmap.sort_values().dropna().iloc[-3:])
# print(heatmap.sort_values())

/tmp/ipykernel_35847/2337464591.py:57: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  backtest = Backtest(set_df_schema(df), System, commission=.001, cash=100000)
